In [1]:
from google.colab import drive
drive.mount('/content/drive')  #listになかったのでドライブ上にデータ保存し、ドライブをマウント。

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold, StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
feat = "feat03"
train = pd.read_csv(f"/content/drive/MyDrive/signate/債務不履行/途中経過_csv/feat_train_{feat}.csv")
test = pd.read_csv(f"/content/drive/MyDrive/signate/債務不履行/途中経過_csv/feat_test_{feat}.csv")

###-----------------------------------------------------------------------------
## カテゴリ変数の抽出
cols_notcat = ['MIS_Status', 'Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob',
               'DisbursementGross', 'GrAppv', 'SBA_Appv','population', 'density',
               'lr_avg_ppi','ApprovalToDisbursement_days','outlier'] # 数値特徴量 'DisbursementGross_minus_SBA_Appv'
# county_col = ['county_gdp','sector_count']
time_col = ["DisbursementDate_timestamp",	"ApprovalDate_timestamp",	"LoanEnd_timestamp"]
dissba = ['DisbursementGross_plus_SBA_Appv','DisbursementGross_minus_SBA_Appv', 'DisbursementGross_times_SBA_Appv','DisbursementGross_divided_by_SBA_Appv']
sbagr = ['SBA_Appv_plus_GrAppv','SBA_Appv_minus_GrAppv', 'SBA_Appv_times_GrAppv','SBA_Appv_divided_by_GrAppv']
disgr = ['DisbursementGross_plus_GrAppv','DisbursementGross_minus_GrAppv', 'DisbursementGross_times_GrAppv','DisbursementGross_divided_by_GrAppv']
# year_col = ["USREC_SUM"]
# urb_col = ["HOUPCT_RUR","ALAND_PCT_RUR","RUR_pct"]
city_col = ["city_Term_mean","city_NoEmp_mean"]

cols_notcat = cols_notcat + city_col + dissba + sbagr + disgr + time_col # 数値特徴量

###-----------------------------------------------------------------------------


cols_cat = [c for c in train.columns if not c in cols_notcat] # カテゴリ特徴量

# ターゲットエンコ変数
target_encoded_cols = [c for c in train.columns if "target_" in c]

ordinal_cols_cat = [c for c in cols_cat if not c in target_encoded_cols]


#-------------------------カテゴリすべて削除------------------------------------

# 削除するカラム(LRはカテゴリ変数を使わずにする)
drop_col = ordinal_cols_cat

# Ordinal Encodingの特徴量を削除
train = train.drop(drop_col, axis=1, errors='ignore')
test = test.drop(drop_col, axis=1, errors='ignore')

In [5]:
### 多重共線性の削除

numeric_cols = train.select_dtypes(include=['number']).columns

# 数値型カラム間の相関行列を計算
corr_matrix = train[numeric_cols].corr()

# 相関係数が0.95を超えるカラムペアを探索
# 相関行列は対称なので、対角線より上（または下）の要素のみを検討すれば十分
high_corr_pairs = []
del_col = []

for i, col1 in enumerate(corr_matrix.columns):
    for j, col2 in enumerate(corr_matrix.index):
        if i < j:  # 対角線より上の要素のみをチェック
            if abs(corr_matrix[col1][col2]) > 0.95:  # 絶対値を取ることで正負の相関も考慮
                high_corr_pairs.append((col1, col2, corr_matrix[col1][col2]))

# 相関係数が0.95を超えるカラムペアの表示
for pair in high_corr_pairs:
    del_col.append(pair[0])
    print(f"{pair[0]} と {pair[1]} の相関係数: {pair[2]:.2f}")

DisbursementGross と GrAppv の相関係数: 0.97
DisbursementGross と DisbursementGross_plus_SBA_Appv の相関係数: 0.99
DisbursementGross と SBA_Appv_plus_GrAppv の相関係数: 0.97
DisbursementGross と DisbursementGross_plus_GrAppv の相関係数: 0.99
GrAppv と SBA_Appv の相関係数: 0.98
GrAppv と DisbursementGross_plus_SBA_Appv の相関係数: 0.99
GrAppv と SBA_Appv_plus_GrAppv の相関係数: 1.00
GrAppv と DisbursementGross_plus_GrAppv の相関係数: 0.99
SBA_Appv と DisbursementGross_plus_SBA_Appv の相関係数: 0.98
SBA_Appv と SBA_Appv_plus_GrAppv の相関係数: 0.99
SBA_Appv と DisbursementGross_plus_GrAppv の相関係数: 0.97
DisbursementGross_plus_SBA_Appv と SBA_Appv_plus_GrAppv の相関係数: 0.99
DisbursementGross_plus_SBA_Appv と DisbursementGross_plus_GrAppv の相関係数: 1.00
DisbursementGross_times_SBA_Appv と SBA_Appv_times_GrAppv の相関係数: 0.99
DisbursementGross_times_SBA_Appv と DisbursementGross_times_GrAppv の相関係数: 0.98
DisbursementGross_divided_by_SBA_Appv と DisbursementGross_divided_by_GrAppv の相関係数: 0.96
SBA_Appv_plus_GrAppv と DisbursementGross_plus_GrAppv の相関係数: 0.99
SBA_Appv_ti

In [6]:
del_col = list(set(del_col))

train = train.drop(del_col, axis=1)
test = test.drop(del_col, axis=1)

In [7]:
cols_exp = [c for c in test.columns]

In [8]:
train

,Term,NoEmp,NewExist,CreateJob,RetainedJob,MIS_Status,population,density,city_Term_mean,city_NoEmp_mean,DisbursementGross_minus_SBA_Appv,SBA_Appv_minus_GrAppv,SBA_Appv_divided_by_GrAppv,DisbursementGross_plus_GrAppv,DisbursementGross_minus_GrAppv,DisbursementGross_times_GrAppv,DisbursementGross_divided_by_GrAppv,lr_avg_ppi,outlier,DisbursementDate_timestamp,ApprovalDate_timestamp,LoanEnd_timestamp,ApprovalToDisbursement_days,target_FranchiseCode,target_RevLineCr,target_LowDoc,target_Sector,target_ApprovalFY,target_city,target_State,target_BankState,target_UrbanRural,target_county_fips
0,163,21,1.0,0,0,1,0.9470,-0.2627,0.34060,0.02557,-0.3486,0.4001,0.6800,-0.3926,-0.0914,-0.24960,-0.2488,0.43580,0.09015,-0.55960,0.84670,0.11224,0.3533,0.919686,0.924868,0.908273,0.940289,0.876591,0.912698,0.908228,0.939937,0.945563,0.895935
1,84,6,1.0,4,0,1,-0.4973,-0.6074,0.27860,0.60200,0.1833,-0.3457,0.3804,0.4482,-0.0914,-0.02534,-0.2488,-0.38040,0.11334,-1.24200,-1.58500,-1.06200,0.4963,0.880468,0.893256,0.907312,0.890526,0.927559,1.000000,0.906921,0.916435,0.946101,1.000000
2,242,45,1.0,4,90,1,-0.4730,1.5840,1.43200,1.92500,-0.2903,0.3508,-1.4180,-0.5920,-0.0578,-0.26560,-0.1290,0.13720,0.02054,0.01506,-0.08124,1.21400,0.4827,0.882409,0.923958,0.908134,0.899727,0.898455,1.000000,0.949468,0.952703,0.825198,0.912693
3,237,4,1.0,0,0,1,-0.0835,-0.5380,0.65670,0.31200,-0.4893,0.5970,1.5790,0.2125,-0.0914,-0.11370,-0.2488,-0.12230,0.11426,0.97800,0.34030,1.81800,0.5330,0.881203,0.923889,0.907469,0.913000,0.903582,0.964435,0.927489,0.941023,0.944465,0.964435
4,184,0,1.0,0,0,1,-0.4504,0.3010,-0.35330,-0.73340,1.0490,-1.5590,0.0807,1.4150,-0.0914,0.54500,-0.2488,-0.62350,0.06290,-2.91200,-0.30960,-1.28500,0.2402,0.880468,0.923693,0.907312,0.939921,0.920891,0.916309,0.883054,0.879085,0.946101,0.919112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42302,283,14,1.0,0,0,1,1.5240,1.1830,0.72300,-0.12720,-0.3486,0.4001,0.6800,-0.3926,-0.0914,-0.24960,-0.2488,0.00975,0.08770,-0.55960,-1.12900,1.19500,0.5190,0.917544,0.923766,0.908344,0.940943,0.874442,0.957404,0.906085,0.883080,0.945499,0.957404
42303,53,2,1.0,0,0,1,3.7520,0.5870,-0.82370,-0.82230,-0.4805,0.5850,0.6800,-0.6973,-0.0914,-0.26830,-0.2488,-0.44400,0.06396,-1.65600,0.96700,-1.62000,0.2452,0.882684,0.816008,0.908344,0.898218,0.826872,0.903226,0.879636,0.941487,0.825253,0.910778
42304,59,6,2.0,0,0,1,0.0575,-0.0993,0.00854,-0.31960,-0.3838,0.4495,0.6800,-0.4739,-0.0914,-0.25780,-0.2488,1.32900,0.10560,0.25500,0.24390,-0.27660,0.4768,0.919606,0.923958,0.908134,0.899727,0.860706,0.928058,0.893253,0.875952,0.944626,0.934211
42305,295,18,1.0,0,8,1,-0.4993,-0.7170,4.19500,0.78960,0.3720,-0.6104,0.0807,0.4766,-0.0914,-0.01334,-0.2488,0.17650,0.02486,-0.58250,-2.07200,1.28900,0.5960,0.881769,0.924868,0.908273,0.897770,0.917582,1.000000,0.808537,0.784226,0.945563,1.000000


Logistic Regression

In [9]:
def train_logistic_regression(train, cols_exp, col_target, params=None):

    if params is None:
        params = {}

    params_add = {}
    params |= params_add

    x = train[cols_exp].to_numpy()
    y = train[col_target].to_numpy()

    # K-fold
    kf = KFold(n_splits=5, shuffle=True, random_state=0)
    y_valid_pred_lst = []
    idx_valid_lst = []
    clf_lst = []

    # cross validation
    for fold, (idx_train, idx_valid) in enumerate(kf.split(x)):
        print("fold", fold)
        x_train = x[idx_train, :]
        x_valid = x[idx_valid, :]
        y_train = y[idx_train]
        y_valid = y[idx_valid]

        # normalization
        scaler = StandardScaler()
        x_train = scaler.fit_transform(x_train)
        x_valid = scaler.transform(x_valid)

        # modeling
        clf = LogisticRegression(random_state=0, max_iter=1000).fit(x_train, y_train)

        # oof
        y_valid_pred = clf.predict_proba(x_valid)
        y_valid_pred_lst.append(y_valid_pred)
        idx_valid_lst.append(idx_valid)
        clf_lst.append(clf)

    idx_valid = np.hstack(idx_valid_lst)
    # y_valid_pred = np.hstack(y_valid_pred_lst)
    y_valid_pred = np.vstack(y_valid_pred_lst)
    oof_pred = y_valid_pred[np.argsort(idx_valid)]

    return clf_lst, oof_pred

In [10]:
def predict_test(x_test, clf_lst):
    y_test_pred_lst = []

    for clf in clf_lst:
        y_test_pred = clf.predict_proba(x_test)
        y_test_pred_lst.append(y_test_pred)

    y_test_pred = np.mean(y_test_pred_lst, axis=0)
    return y_test_pred

In [11]:
col_target = "MIS_Status"
print("col_target =", col_target, "-"*50)

# train logistic regression
clf_lst, oof_pred = train_logistic_regression(train, cols_exp, col_target)

# normalization for test
scaler = StandardScaler()
scaler.fit(train[cols_exp].to_numpy())
x_test = scaler.transform(test[cols_exp].to_numpy())

# predict test with CV ensemble
y_test_pred = predict_test(x_test, clf_lst)

# record
oof_pred_df = pd.DataFrame(oof_pred, columns=[f"MIS_Status_{h}" for h in range(2)])
test_pred_df = pd.DataFrame(y_test_pred, columns=[f"MIS_Status_{h}" for h in range(2)])

col_target = MIS_Status --------------------------------------------------
fold 0
fold 1
fold 2
fold 3
fold 4


In [12]:
from sklearn.metrics import f1_score

# 実際のターゲット値
y_true = train[col_target].values

# OOF予測値をクラスに変換（確率から最も高いクラスを選択）
# 二値分類の場合、0.5を閾値としてクラスを決定することが一般的です
y_oof_pred_class = (oof_pred[:, 1] > 0.5).astype(int)

# mean F1スコア（マクロ平均）を計算
mean_f1_score_macro = f1_score(y_true, y_oof_pred_class, average='macro')
print(f'Mean F1 Score (Macro): {mean_f1_score_macro}')


Mean F1 Score (Macro): 0.626009598390064


In [13]:
# save
oof_pred_df.to_csv(f"/content/drive/MyDrive/signate/債務不履行/model_feat/oof_pred_logisticregression_{feat}.csv", index=False)
test_pred_df.to_csv(f"/content/drive/MyDrive/signate/債務不履行/model_feat/test_pred_logisticregression_{feat}.csv", index=False)

In [14]:
"""
['Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob', 'MIS_Status',
       'SBA_Appv', 'population', 'density', 'city_Term_mean',
       'city_NoEmp_mean', 'county_gdp', 'sector_count',
       'DisbursementGross_minus_SBA_Appv', 'lr_avg_ppi',
       'ApprovalToDisbursement_days', 'target_FranchiseCode',
       'target_RevLineCr', 'target_LowDoc', 'target_Sector',
       'target_ApprovalFY', 'target_city', 'target_State', 'target_BankState',
       'target_county_fips', 'target_App_year', 'target_Disb_year',
       'target_UrbanRural']
#0.600387608395716
"""

"\n['Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob', 'MIS_Status',\n       'SBA_Appv', 'population', 'density', 'city_Term_mean',\n       'city_NoEmp_mean', 'county_gdp', 'sector_count',\n       'DisbursementGross_minus_SBA_Appv', 'lr_avg_ppi',\n       'ApprovalToDisbursement_days', 'target_FranchiseCode',\n       'target_RevLineCr', 'target_LowDoc', 'target_Sector',\n       'target_ApprovalFY', 'target_city', 'target_State', 'target_BankState',\n       'target_county_fips', 'target_App_year', 'target_Disb_year',\n       'target_UrbanRural']\n#0.600387608395716\n"